In [1]:
# Getting the current user,
# in each computer the username is based on the os's user.
# I am using MacBook M1 Pro
# I have created a Folder Aidetic

import os
current_user = os.getenv("USER")
print("Current user is", current_user)

#Current user is neerajvyas

Current user is neerajvyas


In [2]:
# All the libraries 
import pyspark
import importlib
from pyspark.sql import SparkSession
from pyspark.sql.functions import to_date, to_timestamp, col, lower, udf, concat
from pyspark.sql.types import StringType

# FOR MAP CREATION 
#! pip install folium
import folium

In [3]:
# Check if PySpark is installed, if not then we will install pyspark
if importlib.util.find_spec("pyspark") is None:
    print("PySpark is not installed. Installing...")
    !pip install pyspark
else:
    print("PySpark is already installed.")

PySpark is already installed.


#### Creating Pyspark Session

In [4]:
spark = SparkSession.builder.appName("AideticAssignment").getOrCreate()

24/02/23 17:32:34 WARN Utils: Your hostname, Neerajs-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.0.105 instead (on interface en0)
24/02/23 17:32:34 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/02/23 17:32:34 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/02/23 17:32:36 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


### 1. Load the dataset into a PySpark DataFrame.

In [5]:
csv_file_path = "/Users/" + current_user + "/Aidetic/database.csv"
df = spark.read.format("csv").option("header", "true").load(csv_file_path)

In [6]:
df.printSchema()

root
 |-- Date: string (nullable = true)
 |-- Time: string (nullable = true)
 |-- Latitude: string (nullable = true)
 |-- Longitude: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Depth: string (nullable = true)
 |-- Depth Error: string (nullable = true)
 |-- Depth Seismic Stations: string (nullable = true)
 |-- Magnitude: string (nullable = true)
 |-- Magnitude Type: string (nullable = true)
 |-- Magnitude Error: string (nullable = true)
 |-- Magnitude Seismic Stations: string (nullable = true)
 |-- Azimuthal Gap: string (nullable = true)
 |-- Horizontal Distance: string (nullable = true)
 |-- Horizontal Error: string (nullable = true)
 |-- Root Mean Square: string (nullable = true)
 |-- ID: string (nullable = true)
 |-- Source: string (nullable = true)
 |-- Location Source: string (nullable = true)
 |-- Magnitude Source: string (nullable = true)
 |-- Status: string (nullable = true)



In [ ]:
# I am assuming that there are no null values

In [8]:
# Casting the schema accordingly because all are in string format by default.

# Converting string to DateType
# df = df.withColumn("Date", to_date(df["Date"], "MM/dd/yyyy"))

# # Converting string to TimestampType
# df = df.withColumn("Time", to_timestamp(df["Time"], "HH:mm:ss"))

# Converting string to double (Floating type numbers)
df = df.withColumn("Latitude", col("Latitude").cast("double"))
df = df.withColumn("Longitude", col("Longitude").cast("double"))
df = df.withColumn("Depth", col("Depth").cast("double"))
df = df.withColumn("Depth Error", col("Depth Error").cast("double"))
df = df.withColumn("Depth Seismic Stations", col("Depth Seismic Stations").cast("double"))
df = df.withColumn("Magnitude", col("Magnitude").cast("double"))
df = df.withColumn("Magnitude Error", col("Magnitude Error").cast("double"))
df = df.withColumn("Magnitude Seismic Stations", col("Magnitude Seismic Stations").cast("double"))
df = df.withColumn("Azimuthal Gap", col("Azimuthal Gap").cast("double"))
df = df.withColumn("Horizontal Distance", col("Horizontal Distance").cast("double"))
df = df.withColumn("Horizontal Error", col("Magnitude Error").cast("double"))
df = df.withColumn("Root Mean Square", col("Root Mean Square").cast("double"))

In [9]:
df.printSchema()

root
 |-- Date: string (nullable = true)
 |-- Time: string (nullable = true)
 |-- Latitude: double (nullable = true)
 |-- Longitude: double (nullable = true)
 |-- Type: string (nullable = true)
 |-- Depth: double (nullable = true)
 |-- Depth Error: double (nullable = true)
 |-- Depth Seismic Stations: double (nullable = true)
 |-- Magnitude: double (nullable = true)
 |-- Magnitude Type: string (nullable = true)
 |-- Magnitude Error: double (nullable = true)
 |-- Magnitude Seismic Stations: double (nullable = true)
 |-- Azimuthal Gap: double (nullable = true)
 |-- Horizontal Distance: double (nullable = true)
 |-- Horizontal Error: double (nullable = true)
 |-- Root Mean Square: double (nullable = true)
 |-- ID: string (nullable = true)
 |-- Source: string (nullable = true)
 |-- Location Source: string (nullable = true)
 |-- Magnitude Source: string (nullable = true)
 |-- Status: string (nullable = true)



### 2. Convert the Date and Time columns into a timestamp column named Timestamp.


In [10]:
# Date and Time are in String formart and Time stamp is the new column in timestamp format.
df = df.withColumn("Timestamp", to_timestamp(concat(df["Date"], df["Time"]), "MM/dd/yyyyHH:mm:ss"))
df['Date','Time','Timestamp'].show(5)

+----------+--------+-------------------+
|      Date|    Time|          Timestamp|
+----------+--------+-------------------+
|01/02/1965|13:44:18|1965-01-02 13:44:18|
|01/04/1965|11:29:49|1965-01-04 11:29:49|
|01/05/1965|18:05:58|1965-01-05 18:05:58|
|01/08/1965|18:49:43|1965-01-08 18:49:43|
|01/09/1965|13:32:50|1965-01-09 13:32:50|
+----------+--------+-------------------+
only showing top 5 rows



### 3.Filter the dataset to include only earthquakes with a magnitude greater than 5.0.

In [11]:
filtered_df = df.filter((lower(df["Type"]) == "earthquake") & (df["magnitude"] > 5.0))
magintude_greater_than_5 = filtered_df.count()
print(magintude_greater_than_5)
print(df.filter((lower(df["Type"]) == "earthquake") & (df["magnitude"] > 5.0)).count())

23232
23232


### 4. Calculate the average depth and magnitude of earthquakes for each earthquake type

In [12]:
import pyspark.sql.functions as sqlfunc
print("========== Depth Average ============")
df.groupby('Type').agg(sqlfunc.avg('depth')).show()

print("========== Magnitude Average ========")
df.groupby('Type').agg(sqlfunc.avg('magnitude')).show()

========== Depth Average ============
+-----------------+-----------------+
|             Type|       avg(depth)|
+-----------------+-----------------+
|        Explosion|              0.0|
|       Rock Burst|              1.0|
|Nuclear Explosion|              0.3|
|       Earthquake|71.31391348140497|
+-----------------+-----------------+

========== Magnitude Average ========
+-----------------+-----------------+
|             Type|   avg(magnitude)|
+-----------------+-----------------+
|        Explosion|             5.85|
|       Rock Burst|              6.2|
|Nuclear Explosion|5.850685714285718|
|       Earthquake|5.882762568870756|
+-----------------+-----------------+



### 5.Implement a UDF to categorize the earthquakes into levels (e.g., Low, Moderate, High) based on their magnitudes.

In [13]:
df = df.withColumn("Magnitude", col("Magnitude").cast("int"))

In [14]:

# Define a user-defined function to categorize magnitude levels
def classify_magnitude(magnitude):
    if magnitude >= 7:
        return "High"
    elif magnitude >= 5:
        return "Moderate"
    else:
        return "Low"

# Register the Python function as a UDF
classify_magnitude_udf = udf(classify_magnitude, StringType())

# Apply the UDF to the DataFrame
df = df.withColumn("MagnitudeLevel", classify_magnitude_udf(df["Magnitude"]))

In [15]:
df.show()

+----------+--------+--------+---------+----------+-----+-----------+----------------------+---------+--------------+---------------+--------------------------+-------------+-------------------+----------------+----------------+--------------------+---------+---------------+----------------+---------+-------------------+--------------+
|      Date|    Time|Latitude|Longitude|      Type|Depth|Depth Error|Depth Seismic Stations|Magnitude|Magnitude Type|Magnitude Error|Magnitude Seismic Stations|Azimuthal Gap|Horizontal Distance|Horizontal Error|Root Mean Square|                  ID|   Source|Location Source|Magnitude Source|   Status|          Timestamp|MagnitudeLevel|
+----------+--------+--------+---------+----------+-----+-----------+----------------------+---------+--------------+---------------+--------------------------+-------------+-------------------+----------------+----------------+--------------------+---------+---------------+----------------+---------+-------------------+--

### 6. Calculate the distance of each earthquake from a reference location (e.g., (0, 0)).

In [16]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import FloatType
from math import radians, sin, cos, sqrt, atan2

# Reference location
ref_latitude = 0
ref_longitude = 0

# Define Haversine formula as a UDF
@udf(returnType=FloatType())
def calculate_distance(latitude, longitude):
    earth_radius_km = 6371.0  # Earth's radius in kilometers

    # Convert latitude and longitude from degrees to radians
    lat1 = radians(ref_latitude)
    lon1 = radians(ref_longitude)
    lat2 = radians(latitude)
    lon2 = radians(longitude)

    # Calculate differences
    dlat = lat2 - lat1
    dlon = lon2 - lon1

    # Calculate distance using Haversine formula
    a = sin(dlat / 2) ** 2 + cos(lat1) * cos(lat2) * sin(dlon / 2) ** 2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    distance_km = earth_radius_km * c

    return distance_km

# Apply the UDF to calculate distance for each earthquake

df = df.withColumn("Distance_km", calculate_distance(
    df["Latitude"],
    df["Longitude"])
)

# Show DataFrame with distance column

In [17]:
df.show()

+----------+--------+--------+---------+----------+-----+-----------+----------------------+---------+--------------+---------------+--------------------------+-------------+-------------------+----------------+----------------+--------------------+---------+---------------+----------------+---------+-------------------+--------------+-----------+
|      Date|    Time|Latitude|Longitude|      Type|Depth|Depth Error|Depth Seismic Stations|Magnitude|Magnitude Type|Magnitude Error|Magnitude Seismic Stations|Azimuthal Gap|Horizontal Distance|Horizontal Error|Root Mean Square|                  ID|   Source|Location Source|Magnitude Source|   Status|          Timestamp|MagnitudeLevel|Distance_km|
+----------+--------+--------+---------+----------+-----+-----------+----------------------+---------+--------------+---------------+--------------------------+-------------+-------------------+----------------+----------------+--------------------+---------+---------------+----------------+--------

### 7. Visualize the geographical distribution of earthquakes on a world map using appropriate libraries (e.g., Basemap or Folium).

In [18]:
limited_df_distance = df.limit(300)

In [19]:
# Create a Folium map centered at a location
m = folium.Map(location=[0, 0], zoom_start=2)

# df = df.head(100)


# Add markers for each earthquake location
for row in limited_df_distance.collect():
    folium.Marker([row["Latitude"], row["Longitude"]], popup=row["ID"]).add_to(m)

# Save the map to an HTML file
m.save("earthquake_map.html")


### 8.Please include the final csv in the repository.

In [20]:
# Writing the csv file according to Map's Data.
limited_df_distance.coalesce(1).write.csv("map_eqs.csv", header=True)

In [21]:
# Stop SparkSession
spark.stop()